In [32]:
import os
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains 
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import re

page = 1
BASE_URL = 'https://pragativadi.com/'
country = 'bangladesh'

initial_url = f'{BASE_URL}page/{page}/?s={country}'


all_news_links = []

#FOR ALL PAGES REGARDING BANGLADESH, UNCOMMENT THE 'WHILE LOOP' and 'PAGE+=1' COMMAND. REMOVE THE 'FOR LOOP'

#FOR THIS SCRIPT, TAKING LINKS FROM THE FIRST TWO PAGES.

#while True:
for page in range(1,3):

    url = f'{BASE_URL}page/{page}/?s={country}'

    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    all_link_divs = soup.find_all('div', class_ = 'item-inner clearfix')
    
    if len(all_link_divs) > 0:
        
        for each_link_div in all_link_divs:

            all_news_links.append(each_link_div.find('a', class_ = 'post-url post-title').get('href'))
            
    else:
                
        #print(f'Total pages regarding Bangladesh is {page-1}')
        break
        
    #page+=1
    

#SCRAPING ALL NEWS LINKS

counter = 0
data_list = []

for url in all_news_links:
    
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        #TITLE 

        title_tag = soup.find('span',{'itemprop':'headline'})
        title = title_tag.text if title_tag else 'Title not found'
        title_translation = 'None'

        #AUTHOR

        author_tag = soup.find('span', class_ = 'post-author-name')
        author = author_tag.find('b').text if author_tag else 'Author not found'

        #Date

        date_info = soup.find('time', class_ = 'post-published updated')

        if date_info:

            date_data = date_info.get('datetime')

            only_date = date_data.split('T')[0]
            only_time = date_data.split('T')[1]
            time = f"{only_time.split(':')[0]}:{only_time.split(':')[1]}"
            cleaned_date = f"{only_date},{time}"

            source_localtime = datetime.strptime(cleaned_date, "%Y-%m-%d,%H:%M") + timedelta(hours = 5, minutes = 30) 
            bangladesh_localtime = source_localtime + timedelta(minutes=30)

        else:

            date_data = 'Date data not found'

        #NO CONTENT SUMMARIES IN THIS WEBSITE
        content_summary = 'None'
        summary_translation = 'None'

        #CONTENT

        content = []

        main_content_div = soup.find('div', class_ = 'entry-content clearfix single-post-content')

        if main_content_div:

            all_paras = main_content_div.find_all('p')

            for each_para in all_paras:

                content.append(each_para.text)

            full_content = ''.join(content)

        else:

            full_content = 'Content not found'
            
        content_translation = 'None'
            
        
        data_dict = {
            "url": url,
            "title": title,
            "content": full_content,
            "content_summary": content_summary,
            "title_translation":title_translation,
            "content_translation":content_translation,
            "summary translation":summary_translation,
            "author": author,
            "country": country,
            'source_localtime': source_localtime,
            'bangladesh_localtime': bangladesh_localtime

        }

        counter+=1


        if (full_content != "Content Not Found" and title != 'Title not found'):

                if data_dict not in data_list:
                        # Adding to data list
                        data_list.append(data_dict)
                        print(f'Link {counter} added')
        else:
                print(f'Link {counter}')
                print('Skipped due to missing info.')


df = pd.DataFrame(data_list)
df.head()


csv_filename = f"{country}_Pragativadi.csv"

# Checking if the CSV file already exists
if os.path.exists(csv_filename):
    existing_df = pd.read_csv(csv_filename)
    # Merging new and existing dataframe
    df = pd.concat([existing_df, pd.DataFrame(data_list)], ignore_index=True)
    df["bangladesh_localtime"] = pd.to_datetime(df["bangladesh_localtime"])  # Converting the "date" column to datetime
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.sort_values(by="date", ascending=False)  # Sorting the date
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)  
else:
    # If csv file does not exist, then we create a new CSV file with the scraped data
    df = pd.DataFrame(data_list)
    df = df.sort_values(by="bangladesh_localtime", ascending=False)
    df = df.drop_duplicates(subset=["title"], keep="first")
    df = df.reset_index(drop=True)
    df.to_csv(csv_filename, index=False)

Link 1 added
Link 2 added
Link 3 added
Link 4 added
Link 5 added
Link 6 added
Link 7 added
Link 8 added
Link 9 added
Link 10 added
Link 11 added
Link 12 added
Link 13 added
Link 14 added
Link 15 added
Link 16 added
Link 17 added
Link 18 added
Link 19 added
Link 20 added


,url,title,content,content_summary,title_translation,content_translation,summary translation,author,country,source_localtime,bangladesh_localtime
0,https://pragativadi.com/youth-delegation-from-...,Youth Delegation From Bangladesh Calls On Pres...,New Delhi: A youth delegation from Bangladesh ...,None,None,None,None,Yajati Keshari Rout,bangladesh,2024-02-28 04:23:00,2024-02-28 04:53:00
1,https://pragativadi.com/s-jaishankar-calls-on-...,"S. Jaishankar calls on Sheikh Hasina, Apprecia...","The External Affairs Minister, Dr S. Jaishanka...",None,None,None,None,Yajati Keshari Rout,bangladesh,2024-02-18 05:00:00,2024-02-18 05:30:00
2,https://pragativadi.com/bangladesh-pm-sheikh-h...,Bangladesh PM Sheikh Hasina Wins 5th Term,Dhaka: Bangladesh’s Prime Minister Sheikh Hasi...,None,None,None,None,Ananya Pattnaik,bangladesh,2024-01-08 16:07:00,2024-01-08 16:37:00
3,https://pragativadi.com/bangladesh-massive-fir...,Bangladesh: Massive Fire Breaks Out In Refugee...,Dhaka: A massive fire swept through a Rohingya...,None,None,None,None,Itishree Sethy,bangladesh,2024-01-08 15:57:00,2024-01-08 16:27:00
4,https://pragativadi.com/pm-hasina-likely-to-wi...,PM Hasina Likely To Win 4th Straight Term As B...,Dhaka: Voting for the 12th general election in...,None,None,None,None,Ananya Pattnaik,bangladesh,2024-01-07 15:13:00,2024-01-07 15:43:00
